In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [4]:
def Radius_tilla(x, y, s2):
    return np.sqrt(x**2 + s2*y*y)
    
def Modul(v):
    return np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)

def Normalize(v):
    return v/Modul(v)
  
def F(radi, c, k):
    return -k * np.exp(- c * radi)

def D_F (z, c):
    return -c * z
    
def D_D_F (z, c):
    return c * c * z

def V_r (x, v, s2):
    return (x[0]*v[0] + s2*x[1]*v[1])/Radius_tilla(x[0], x[1], s2)
    
def V_z (x, v, c, s2):
    return D_F(x[2], c) * V_r (x, v, s2)
    
def Normal(x, v, c, k, s2):
    r = Radius_tilla(x[0], x[1], s2)
    v_r = (x[0]*v[0] + s2*x[1]*v[1])/r
    dF = D_F(F(r,c,k),c)
    ddF = D_D_F(F(r,c,k),c)
    
    #cÃ lcul i normalitzaciÃ³ del vector normal a la superfÃ­cie
    n = np.array([-(dF/r) * x[0],-(dF/r) * x[1]*s2, 1])
    modul_n = Modul(n)
    n = n / modul_n          #Normalitzem el vector normal  
    
    #Component normal de l'acceleraciÃ³
    a_n = ((ddF*r - dF)*v_r**2 + dF*(v[0]**2 + s2*v[1]*v[1]))/(r * modul_n) 

    #mÃ²dul i vector de forÃ§a normal
    m_normal = m*a_n + m*g/modul_n
    normal = m_normal * n
    
    return m_normal, normal
    
def Friction(v, N, mu):
    u_v = v / Modul(v) #Ã©s el vector velocitat unitari
    return - mu * N * u_v

def Acc(x, v, c, k, s2, mu):
    N, normal = Normal(x, v, c, k, s2) #mÃ²dul i vector
    gravity = np.array([0., 0., -m*g])
    friction = Friction(v, N, mu)
    return normal + gravity + friction
    
    
def E_kinetic(v):
    v_square = v[0]**2 + v[1]**2 + v[2]**2
    return m * v_square/2.
    
def E_potential(x):
    return m * g * x[2]
g = 9.81
m = 1.

def complete_show (tableau):
    fig = plt.figure('pos-accel-time')
    axe = fig.gca(projection='3d')
    axe.plot(tableau['x'],tableau['y'],tableau['t'], label='trajectÃ²ria')
    axe.plot(tableau['ax'],tableau['ay'],tableau['t'], label='acceleraciÃ³')
    axe.plot(tableau['axe'],tableau['aye'],tableau['t'], label='accel_Ã¨bola')
    plt.xlabel('x var')
    plt.ylabel('y var')
    axe.set_zlabel('time')

    axe.legend()
    plt.show()

def distancia_acc (tableau, plot = 0):
    difx = pd.Series.as_matrix(tableau['ax']-tableau['axe'])
    dify = pd.Series.as_matrix(tableau['ay']-tableau['aye'])
    if (plot == True):
        plt.plot(tableau['t'], difx, label = 'x')
        plt.plot(tableau['t'], dify, label = 'y')
        plt.legend()
    return (np.dot(difx,difx) + np.dot(dify,dify))/len(tableau['t'])

def calcula(taula, c, k, sigma, mu):
    k = [ebola2r(c,k,sigma,mu,taula["x"][i],taula["y"][i],taula["vx"][i],taula["vy"][i]) for i in range(len(taula))]
    k = np.array(k)
    taula["axe"] = k[:,0]
    taula["aye"] = k[:,1]
    return taula


def ebola2r(c, k, sigma, mu, x, y, vx, vy):
    #sigma:
    s2 = sigma**2
    pos = [x, y, F(Radius_tilla(x, y, s2), c, k)]
    vel = [vx, vy, 0]
    vel[2] = V_z(pos, vel,c,s2)
    acc = Acc(pos,vel,c,k,s2,mu)
    return acc

In [9]:
from os import listdir
listdir()

['.ipynb_checkpoints',
 '3.1.2-tmax = 10.0, timestep = 0.004, ws = 0.0, sigma = 1.0',
 '31615880951-604093847-ticket.pdf',
 'Aprenent a integrar.ipynb',
 'Big data and Bias in the web (Ricardo Baeza-Yates).txt',
 'Bitllets València 1.pdf',
 'Bitllets València 2.pdf',
 'dataframe_primers_results.txt',
 'desktop.ini',
 'drive-download-20170221T193914Z.zip',
 'ebola.py',
 'ebola2 dades',
 'ebola2.py',
 'EBOLA2OPTFINAL.ipynb',
 'EBOLA2OPTFINAL.py',
 'Electromagnetism.zip',
 'Eventbrite-Tools',
 'ffmpeg-20170225-7e4f32f-win64-static.zip',
 'ffmpeg-3.2.4.tar.bz2',
 "Fisidabo Febrer '17",
 'FISIDABO+API.pdf',
 'Fisidabo-API',
 'go.txt',
 'Importing Notebooks.ipynb',
 'Introduction to functional programming.txt',
 "La integral de l'Adam returns.ipynb",
 "La integral de l'adam.ipynb",
 'Llibres de Consulta',
 'Memòria fixa Q4',
 'offset.zip',
 'opteb2.py',
 'Optimitzant+l%27ebola2.html',
 'Prova AWS.ipynb',
 'prova.csv',
 'raw_results.txt',
 'Sense títol.jpg',
 'spam de lapi',
 "Takes de l'èbol

In [5]:
#Previ import dels datasets via dropbox

In [7]:
datasets = [pd.read_csv("\Fisidabo-API\Dades_polinomis\take"+str(i)+"_hr.csv") for i in range(1,5)]

OSError: File b'\\Fisidabo-API\\Dades_polinomis\take1_hr.csv' does not exist

In [ ]:
lims = [c_l,c_u,k_l,k_u,sigma_l,sigma_u,mu_l,mu_u] = [1,2.5,0.1,0.4,1.2,2.5,0.002,0.015]

In [ ]:
def fitness(v, dades_take):
    #[c,k,sigma,mu] = v
    return distancia_acc(calcula(dades_take, *v))
def fitness_avg(v,dades_take1 = datasets[0], dades_take2 = datasets[1], dades_take3 = datasets[2]):
    return (fitness(v,dades_take1) + fitness(v,dades_take2) + fitness(v,dades_take3))*(1/3)
def fit(v):
    return fitness_avg(v)
def constrc1(v):
    return v[0]-c_l
def constrc2(v):
    return c_u-v[0]
def constrk1(v):
    return v[1]-k_l
def constrk2(v):
    return k_u-v[1]
def constrsigma1(v):
    return v[2]-sigma_l
def constrsigma2(v):
    return sigma_u-v[2]
def constrmu1(v):
    return v[3]-mu_l
def constrmu2(v):
    return mu_u-v[3]
constr = [constrc1, constrc2, constrk1, constrk2, constrsigma1, constrsigma2, constrmu1, constrmu2]
consi =  []
for i in constr:
    consi.append({'type': 'ineq', 'fun': i})

In [ ]:
resultats = []
i = 0
for c in np.linspace(c_l,c_u,5):
    for k in np.linspace(k_l,k_u,5):
        for sigma in np.linspace(sigma_l,sigma_u,5):
            for mu in np.linspace(mu_l,mu_u,5):
                i += 1
                xinicial = [c, k, sigma, mu]
                t0 = time.clock()
                a = minimize(fitness1, xinicial, method='Nelder-Mead', options={'disp': True, 'maxiter': 100,'maxfev': 50})
                t1 = time.clock()
                print("*"+str(i)+"/"+str(5**4), " xinicial ", xinicial, " Ã²ptim ", a.x, " funcio ", a.fun, " temps ", t1-t0)
                resultats.append([xinicial, a.x, a.fun])

In [2]:
resultats

NameError: name 'resultats' is not defined

In [98]:
po = pd.DataFrame(resultats, columns = ["x_inicial", "x_final", "fitness"])

In [95]:
po[["x_inicial", "x_final"]][po['fitness'] <= 1.1*po['fitness'].min()]['x_final']

11     [1.54786174442, 0.202636150713, 1.02780142762,...
12     [1.54786174442, 0.202636150713, 1.02780142762,...
13     [1.54786174442, 0.202636150713, 1.02780142762,...
14     [1.54786174442, 0.202636150713, 1.02780142762,...
26     [1.22261097074, 0.239726426833, 1.01494410884,...
27     [1.24134079139, 0.238378888395, 1.00402793203,...
28     [1.21879265941, 0.241807998049, 1.00240295493,...
29     [1.2289381674, 0.242315829346, 1.00145615069, ...
31     [1.21823222004, 0.239672096157, 1.01534023128,...
32     [1.19954854852, 0.248488408632, 1.00535782321,...
33     [1.21493533613, 0.240899331761, 1.02290785662,...
34     [1.22239699511, 0.240260818827, 0.988607018119...
36     [1.07520866853, 0.259900027072, 1.03455384672,...
37     [1.08353723532, 0.256694248077, 1.03288353834,...
38     [1.08353723532, 0.256694248077, 1.03288353834,...
39     [1.02428708323, 0.276894356526, 0.991378908605...
41     [1.36516865091, 0.222001579063, 0.976990799429...
42     [1.3578883184, 0.2268397

In [1]:
po['x_final'][53]

NameError: name 'po' is not defined

In [88]:
fitness1

<function __main__.fitness1>

In [91]:
fitness1(po['x_final'][53])

0.099081887119500819